## **Data Cleaning**

### breve descritivo sobre o que se trata

In [49]:
import pandas as pd

In [50]:
base = pd.read_excel("ChavesClientes.xlsx",sheet_name = "base")

In [51]:
base.head(10)

,ID,ChaveSituacao,ClassRisco,CatCliente,Pagamento
0,1,32FC,Ccinza,Basic-Alpha,1
1,2,25MV,AAmarelo,Black,1
2,3,27MV,B-Amarelo,Basic-Beta,1
3,4,26FD,BAmarelo,Black,0
4,5,26FD,C-Amarelo,Black,0
5,6,28FC,C-Amarelo,Platinum-Alpha,0
6,7,27MD,A-Verde,Platinum-Beta,1
7,8,31MD,C-Cinza,Basic,0
8,9,28FS,A-Cinza,Black,1
9,10,31MV,C+Amarelo,Platinum,1


**Verificando a cardinalidade desses dados**

In [52]:
base.groupby(["Pagamento","ChaveSituacao"])["Pagamento"].count()

Pagamento  ChaveSituacao
0          26FD             2
           28FC             2
           28MD             1
           30FC             1
           31MD             1
1          25FD             1
           25FV             1
           25MV             1
           26MC             2
           27MC             1
           27MD             2
           27MV             1
           28FS             1
           29MV             1
           31MV             1
           32FC             1
Name: Pagamento, dtype: int64

### **REGRAS DE NEGÓCIO**
Ao analisar essa base, percebemos que existe uma alta cardinalidade e por isso precisamos tratar esses dados

- Representação de cada coluna:
    - ChaveSituacao:
        - Idade do cliente (a idade mínima para ser cliente é 18 anos)
        - Gênero do cliente:
            - M: Masculino
            - F: Feminino
        - Estado civil do cliente:
            - S: solteiro
            - C: casado
            - D: divorciado
            - V: viúvo
    - ClassRisco:
        - Classificação do cliente como (A,B,C) e indicador (+,- ou vazio)
        - Cor do cliente de acordo com um modelo de churn interno da empresa
    - CatCliente:
        - Categoria do cartão: qual o tipo de cartão do cliente:
            - Basic
            - Black
            - Platinum
        - Categoria VIP: categoria do cliente VIP (caso exista)
            - Alpha
            - Beta

**Podemos começar tratando simplesmente separando a string da coluna**

In [53]:
texto = '32FC'

In [54]:
texto[2:3]

'F'

In [55]:
base['Idade'] = base.ChaveSituacao.str[:2]
base['Genero'] = base.ChaveSituacao.str[2:3]
base['EstadoCivil'] = base.ChaveSituacao.str[-1]

In [56]:
base.head()

,ID,ChaveSituacao,ClassRisco,CatCliente,Pagamento,Idade,Genero,EstadoCivil
0,1,32FC,Ccinza,Basic-Alpha,1,32,F,C
1,2,25MV,AAmarelo,Black,1,25,M,V
2,3,27MV,B-Amarelo,Basic-Beta,1,27,M,V
3,4,26FD,BAmarelo,Black,0,26,F,D
4,5,26FD,C-Amarelo,Black,0,26,F,D


In [57]:
base.groupby(["Pagamento","EstadoCivil"])["Pagamento"].count()

Pagamento  EstadoCivil
0          C              3
           D              4
1          C              4
           D              3
           S              1
           V              5
Name: Pagamento, dtype: int64

**Podemos também fazer o split (divisão) de um valor baseado em um delimitador**

In [58]:
texto = 'Basic-Alpha'

In [59]:
texto.split('-') 

['Basic', 'Alpha']

In [60]:
base['Categoria'] = base.CatCliente.str.split('-').str.get(0)
base['CatVIP'] = base.CatCliente.str.split('-').str.get(1)

In [61]:
base.head(20)

,ID,ChaveSituacao,ClassRisco,CatCliente,Pagamento,Idade,Genero,EstadoCivil,Categoria,CatVIP
0,1,32FC,Ccinza,Basic-Alpha,1,32,F,C,Basic,Alpha
1,2,25MV,AAmarelo,Black,1,25,M,V,Black,NaN
2,3,27MV,B-Amarelo,Basic-Beta,1,27,M,V,Basic,Beta
3,4,26FD,BAmarelo,Black,0,26,F,D,Black,NaN
4,5,26FD,C-Amarelo,Black,0,26,F,D,Black,NaN
5,6,28FC,C-Amarelo,Platinum-Alpha,0,28,F,C,Platinum,Alpha
6,7,27MD,A-Verde,Platinum-Beta,1,27,M,D,Platinum,Beta
7,8,31MD,C-Cinza,Basic,0,31,M,D,Basic,NaN
8,9,28FS,A-Cinza,Black,1,28,F,S,Black,NaN
9,10,31MV,C+Amarelo,Platinum,1,31,M,V,Platinum,NaN


**Caso a gente não consiga separar pela posição do texto ou por um delimitador, ainda temos a opção de usar o Regex**

In [62]:
import re

In [63]:
re.findall('Teste','meu nome é Teste')

['Teste']

- Regex:
    - ^: Começa com
    - $: Termina com
    - *: O último caracter repetido 0 ou mais vezes
    - +: O último caracter repetido 1 ou mais vezes
    - ?: O último caracter repetido 0 ou 1 vez
    - [A-Z]: qualquer valor em maiúsculo

In [64]:
# Buscando Lucas no texto abaixo
re.findall('Te*ste','meu nome é Tste')

['Tste']

In [65]:
# Verificando se começa ou termina com um caracter (usando ˆ, $)
re.findall('^meu','meu nome é Teste')

['meu']

In [66]:
# Verificando se existe parte da palavra Lucas (usando *, +, ?)
re.findall('Te+ste','meu nome é Teste')

['Teste']

In [67]:
# Buscando por letras maiúsculas
re.findall('[A-Z]+\w{4}','meu nome é Teste')

['Teste']

In [68]:
base.head()

,ID,ChaveSituacao,ClassRisco,CatCliente,Pagamento,Idade,Genero,EstadoCivil,Categoria,CatVIP
0,1,32FC,Ccinza,Basic-Alpha,1,32,F,C,Basic,Alpha
1,2,25MV,AAmarelo,Black,1,25,M,V,Black,NaN
2,3,27MV,B-Amarelo,Basic-Beta,1,27,M,V,Basic,Beta
3,4,26FD,BAmarelo,Black,0,26,F,D,Black,NaN
4,5,26FD,C-Amarelo,Black,0,26,F,D,Black,NaN


In [69]:
# Buscando agora a classificação do cliente dentro da coluna "ClassRisco"
re.findall('^[A-Z][^A-Za-z]?','AAmarelo')

['A']

In [70]:
base['Risco'] = base.ClassRisco.apply(lambda x: re.findall('^[A-Z][^A-Za-z]?',x)[0])

In [71]:
base.head(20)

,ID,ChaveSituacao,ClassRisco,CatCliente,Pagamento,Idade,Genero,EstadoCivil,Categoria,CatVIP,Risco
0,1,32FC,Ccinza,Basic-Alpha,1,32,F,C,Basic,Alpha,C
1,2,25MV,AAmarelo,Black,1,25,M,V,Black,NaN,A
2,3,27MV,B-Amarelo,Basic-Beta,1,27,M,V,Basic,Beta,B-
3,4,26FD,BAmarelo,Black,0,26,F,D,Black,NaN,B
4,5,26FD,C-Amarelo,Black,0,26,F,D,Black,NaN,C-
5,6,28FC,C-Amarelo,Platinum-Alpha,0,28,F,C,Platinum,Alpha,C-
6,7,27MD,A-Verde,Platinum-Beta,1,27,M,D,Platinum,Beta,A-
7,8,31MD,C-Cinza,Basic,0,31,M,D,Basic,NaN,C-
8,9,28FS,A-Cinza,Black,1,28,F,S,Black,NaN,A-
9,10,31MV,C+Amarelo,Platinum,1,31,M,V,Platinum,NaN,C+


In [41]:
base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ID             20 non-null     int64 
 1   ChaveSituacao  20 non-null     object
 2   ClassRisco     20 non-null     object
 3   CatCliente     20 non-null     object
 4   Pagamento      20 non-null     int64 
 5   Idade          20 non-null     object
 6   Genero         20 non-null     object
 7   EstadoCivil    20 non-null     object
 8   Risco          20 non-null     object
 9   Categoria      20 non-null     object
 10  CatVIP         8 non-null      object
dtypes: int64(2), object(9)
memory usage: 1.8+ KB


**Transformando a coluna 'Idade' em numérico**

In [42]:
base['Idade'] = pd.to_numeric(base['Idade'])

In [43]:
base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ID             20 non-null     int64 
 1   ChaveSituacao  20 non-null     object
 2   ClassRisco     20 non-null     object
 3   CatCliente     20 non-null     object
 4   Pagamento      20 non-null     int64 
 5   Idade          20 non-null     int64 
 6   Genero         20 non-null     object
 7   EstadoCivil    20 non-null     object
 8   Risco          20 non-null     object
 9   Categoria      20 non-null     object
 10  CatVIP         8 non-null      object
dtypes: int64(3), object(8)
memory usage: 1.8+ KB


**Por fim, tratando os valores vazios**

In [44]:
base.loc[base.CatVIP.isnull(),"CatVIP"]="Comum"

In [48]:
base.head(20)

,ID,ChaveSituacao,ClassRisco,CatCliente,Pagamento,Idade,Genero,EstadoCivil,Risco,Categoria,CatVIP
0,1,32FC,Ccinza,Basic-Alpha,1,32,F,C,C,Basic,Alpha
1,2,25MV,AAmarelo,Black,1,25,M,V,A,Black,Comum
2,3,27MV,B-Amarelo,Basic-Beta,1,27,M,V,B-,Basic,Beta
3,4,26FD,BAmarelo,Black,0,26,F,D,B,Black,Comum
4,5,26FD,C-Amarelo,Black,0,26,F,D,C-,Black,Comum
5,6,28FC,C-Amarelo,Platinum-Alpha,0,28,F,C,C-,Platinum,Alpha
6,7,27MD,A-Verde,Platinum-Beta,1,27,M,D,A-,Platinum,Beta
7,8,31MD,C-Cinza,Basic,0,31,M,D,C-,Basic,Comum
8,9,28FS,A-Cinza,Black,1,28,F,S,A-,Black,Comum
9,10,31MV,C+Amarelo,Platinum,1,31,M,V,C+,Platinum,Comum


In [72]:
base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ID             20 non-null     int64 
 1   ChaveSituacao  20 non-null     object
 2   ClassRisco     20 non-null     object
 3   CatCliente     20 non-null     object
 4   Pagamento      20 non-null     int64 
 5   Idade          20 non-null     object
 6   Genero         20 non-null     object
 7   EstadoCivil    20 non-null     object
 8   Categoria      20 non-null     object
 9   CatVIP         8 non-null      object
 10  Risco          20 non-null     object
dtypes: int64(2), object(9)
memory usage: 1.8+ KB


**Agora podemos criar uma nova base apenas com as colunas que acabamos de criar**

In [74]:
base.groupby(["Pagamento","CatVIP"])["Pagamento"].count()

Pagamento  CatVIP
0          Alpha     1
           Beta      2
1          Alpha     3
           Beta      2
Name: Pagamento, dtype: int64